In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import os

if os.getcwd().split("/")[-1] != "Road-Segmentation-ML":
    os.chdir("..")
print("CWD:", os.getcwd())

In [ ]:
import torch
from omegaconf import OmegaConf
import segmentation_models_pytorch as smp
from train_utils import prepare_data, prepare_model, prepare_optimizer, train, set_seeds

In [5]:
# set seeds
set_seeds()

# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [ ]:
args = OmegaConf.create(
    dict(
        # General
        seed=0,
        # Data folders
        image_folders=["datasets/train/images/"],
        gt_folders=["datasets/train/groundtruth/"],
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        input_size=(384),  # resize
        random_resized_crop_scale=(0.5, 0.5),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=True,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=True,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=False,
        degrees=5,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=False,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=True,  # TODO: Should it always be True?
        # Data loaders
        batch_size=2,
        train_size=0.8,
        val_size=0.2,
        # Model
        # UNetV1
        model_name="UNetV3",  # Change for v3
        model_save_name="models/checkpoints/unetv3-test.pt",
        # Optimizer
        optim_name="adam",  # sgd
        optim_lr=0.001,
        optim_momentum=0.9,  # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=2000,
        eval_freq=100,
        # Wandb logging
        wandb_project="road-segmentation",
        wandb_run="unet-v3",
        entity="feeit",
    )
)

In [ ]:
# prepare train and validation loaders
train_loader, val_loader = prepare_data(args)

In [ ]:
# prepare model
model = prepare_model(args)
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()
# criterion = DiceLoss()
# prepare optimizer
optimizer = prepare_optimizer(model, args)

In [ ]:
# device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# train
trained_model = train(
    model, device, train_loader, val_loader, criterion, optimizer, args
)

In [ ]:
# from train_utils import batch_mean_and_sd

# # calculate mean and std of the dataset
# mean, std = batch_mean_and_sd(train_loader)

Submission

In [6]:
import torch
import torchvision.transforms.v2 as transforms
import torchvision.models.segmentation as models
import matplotlib.pyplot as plt
from datasets.TestDataset import TestDataset
from models.UNetV3 import UNetV3
from examples.mask_to_submission import *

In [7]:
# select checkpoint
MODEL = "models/checkpoints/unetv3-3_effnet.pt"
# load checkpoint
try:
    checkpoint = torch.load(MODEL)
except:
    print("Loading checkpoint failed. Trying to load it with map_location.")
    checkpoint = torch.load(MODEL, map_location=torch.device("cpu"))
# create model
model = UNetV3()
# load model weights
model.load_state_dict(checkpoint)

Loading checkpoint failed. Trying to load it with map_location.


<All keys matched successfully>

In [8]:
# path to the test folder
test_folder = "datasets/test/"

# set mean and std
std = [0.1967, 0.1896, 0.1897]
means = [0.3353, 0.3328, 0.2984]

# define transformations
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(means, std=std),
    ]
)

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

/home/nadezhda/.local/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [ ]:
# create folder predictions if it doesn't exist
if not os.path.exists("predictions"):
    os.makedirs("predictions")
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # threshold prediction
    prediction = torch.sigmoid(prediction)
    prediction = (prediction > 0.5).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

In [ ]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)

Post Processing

In [ ]:
import cv2
import numpy as np


def apply_morphological_ops(segmentation_map):
    # Define a kernel for operations (3x3 square)
    kernel = np.ones((3, 3), np.uint8)

    # Apply erosion
    erosion = cv2.erode(segmentation_map, kernel, iterations=4)

    # Apply dilation
    dilation = cv2.dilate(erosion, kernel, iterations=4)

    return dilation


# Assuming 'segmentation_map' is your initial segmentation result as a binary image
# processed_map = apply_morphological_ops(segmentation_map)

In [ ]:
def postprocess_segmentation(image):
    # Load the image (assuming it's a binary image with roads as white and background as black)

    # Noise reduction (using morphological opening)
    kernel_op = np.zeros((5, 5), np.uint8)
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel_op, iterations=3)

    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_op, iterations=3)

    # kernel_dil = np.zeros((3, 3), np.uint8)
    # Background area determination (Dilation to increase the background area)
    # sure_bg = cv2.dilate(opening, kernel_dil, iterations=3)

    # # Identifying sure foreground area (roads)
    # dist_transform = cv2.distanceTransform(opening, cv2.DIST_L1, 5)
    # _, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)= 0

    return closing